## Contents

In the following sections, we develop the theoretical foundations and key computations for each stage of our analysis:

1. **Clustering Pipeline**  
2. **Co-activation Analysis & Relative Coherence**  
3. **SAE Match**  
4. **Evolution of Clusters via SAE Match**  
5. **Evolution of Clusters via Optimal Transport**  

Each chapter combines concise theory, essential formulas, and methodological insights to support the corresponding implementation modules.

---

# 1 Clustering Pipeline

This project adopts a **“UMAP → HDBSCAN” pipeline** to group Sparse-Autoencoder (SAE) features into semantically coherent clusters.  
After experiments showed that clustering the **decoder weight columns** yields poor intra-cluster coherence, the analysis focuses on clustering **embedded textual descriptions** of each latent (from Neuronpedia), which produces markedly better structure.

---

## 1.1 Stage A – UMAP Manifold Projection

**Objective:** map high-dimensional vectors—either decoder columns $\{\mathbf v_i\in\mathbb R^{768}\}$ or sentence embeddings $\{\mathbf e_i\in\mathbb R^{768}\}$ — into $\{\mathbf z_i\in\mathbb R^{15}\}$ so that close points remain neighbors and global structure is retained.

1. **Graph construction**  
   - Compute cosine distances $d_{ij}$ between all pairs.  
   - For each $i$, select `k` nearest neighbours (`n_neighbors = k`) and assign fuzzy weights $p_{ij}$.

2. **Low-D layout**  
   - Initialise $\mathbf z_i\in\mathbb R^{15}$ randomly.  
   - Minimise the cross-entropy between high-dimensional affinities $p_{ij}$ and low-dimensional affinities  
     $$  
       q_{ij} = \frac{1}{1 + a\,\|\mathbf z_i-\mathbf z_j\|_2^{2b}}.  
     $$  
   - Loss:  
     $$  
       \mathcal L  
       = -\sum_{i<j}\bigl[p_{ij}\log q_{ij} + (1-p_{ij})\log(1-q_{ij})\bigr].  
     $$

3. **Hyperparameters**  
   - **`n_neighbors = k`** $\in\{10,20\}$: small $k$ → very local; large $k$ → more global.  
   - **`min_dist`** $\in\{0.1,0.05\}$: controls how tightly points pack in $\mathbb R^{15}$.

We grid-search $(k,\text{min\_dist})$ to maximise downstream cluster coherence.  


---

## 1.2 Stage B – HDBSCAN Density Clustering

Starting from the UMAP embedding $\{\mathbf z_i\in\mathbb R^{15}\}$, we run HDBSCAN (Euclidean) in two passes:

1. **Primary grid search**  
   - **UMAP parameter**: `n_neighbors = k` ∈ {10, 20}  
   - **HDBSCAN parameter**: `min_cluster_size = m` ∈ {8, 12, 20}  
   - For each (k,m) pair:  
     1. Recompute $\mathbf z_i$ with UMAP(n_neighbors=k).  
     2. Cluster with HDBSCAN(min_cluster_size=m).  
     3. Evaluate **mean coherence** on the resulting clusters.  
   - **Select** the (k,m) that maximizes mean coherence.

2. **Noise-refinement**  
   - Collect points labeled “noise” (cluster = –1) from the primary pass.  
   - **Refine UMAP** with `n_neighbors ∈ {5, 10, 15}` and **HDBSCAN** with `min_cluster_size ∈ {3, 4, 6}`.  
   - **Accept** any newly formed cluster only if its mean coherence ≥ 0.20; otherwise those points remain noise.

---

## 1.3 Cluster-Quality Criteria

We use **sentence embeddings** $\{\mathbf e_i\!\in\!\mathbb R^{768}\}$ for Mean coherence, and **UMAP embeddings** $\{\mathbf z_i\!\in\!\mathbb R^{15}\}$ for Silhouette & Davies–Bouldin.

**Mean coherence**  
$$
\mathrm{coh}(C)
=\frac{2}{|C|(|C|-1)}
\sum_{i<j\in C}
\frac{\mathbf e_i^\top \mathbf e_j}{\|\mathbf e_i\|_2\,\|\mathbf e_j\|_2}.
$$
— Mean coherence measures the average pairwise cosine similarity among all items in a cluster, quantifying how semantically tight the cluster is. It is the **primary** metric for our clusters based on embeddings of text descriptions. 

**Silhouette**  
For $i\in C_j$ in $\mathbf z$-space:
\begin{aligned}
a(i)&=\frac1{|C_j|-1}\sum_{k\in C_j,k\neq i}\|\mathbf z_i-\mathbf z_k\|_2,\\
b(i)&=\min_{m\neq j}\frac1{|C_m|}\sum_{\ell\in C_m}\|\mathbf z_i-\mathbf z_\ell\|_2,\\
s(i)&=\frac{b(i)-a(i)}{\max\{a(i),b(i)\}},\quad
\mathrm{Silhouette}=\frac1{N'}\sum_{\text{non-noise}}s(i).
\end{aligned}

— Silhouette measures for each point the relative difference between its average distance to points in its own cluster and to points in the nearest other cluster, indicating how well it fits its assigned group.

**Davies–Bouldin**  
Let  
$$
\mu_j=\frac1{|C_j|}\sum_{i\in C_j}\mathbf z_i,\quad
S_j=\frac1{|C_j|}\sum_{i\in C_j}\|\mathbf z_i-\mu_j\|_2, \qquad \text{then} \qquad R_{jk}=\frac{S_j+S_k}{\|\mu_j-\mu_k\|_2},\quad
\mathrm{DB}=\frac1K\sum_{j=1}^K\max_{k\neq j}R_{jk}.
$$

— Davies–Bouldin index measures the average ratio of each cluster’s internal scatter to its separation from the most similar other cluster, with lower values indicating better clustering.

**Noise ratio**  
$$
\frac{|\{i:\mathrm{label}(i)=-1\}|}{N}.
$$

— Noise ratio measures the proportion of points labeled as noise (unassigned to any cluster), indicating how much data the algorithm considers outliers.

---

## 1.4 Why Text-Embedding Clustering Wins  

| Variant | Observed mean coherence | Interpretation |
|---------|------------------------|----------------|
| **Decoder weights** | $\approx 0.05$ | Geometric neighbours in weight space rarely share semantics. |
| **Sentence-embeddings** | $\gtrsim 0.40$ after refinement | Captures lexical & contextual similarity → semantically tidy clusters.|

Accordingly, subsequent analyses (SAE Match, OT evolution, etc.) are built exclusively on **description-based clusters**.

---

## 1.5 Reproducibility & Storage  

* Global seed = 42 for `random`, `numpy`, `torch`, and UMAP/HDBSCAN.  
* Results saved per layer: CSV of unique descriptions, full label vector (`–2` = no description, `–1` = noise), metrics JSON, and reduced coordinates `.npy`.

---

### Take-away

The two-stage UMAP → HDBSCAN pipeline, coupled with a targeted noise-refinement pass and coherence-driven model selection, delivers high-quality clusters of SAE feature descriptions — an essential substrate for all further interpretability analyses in this project.

---

# 2 Co-activation Analysis & Relative Coherence

In this section, we capture how SAE latent features co-fire on a subsampled token stream by building a binary activation matrix. We then compute pairwise co-occurrence measures (Jaccard index and Normalized PMI) and aggregate them into cluster-level coherence scores. To evaluate significance, each cluster’s mean metric is compared against randomized baselines, producing the **relative Jaccard** and **relative NPMI** that quantify how much more coherently features fire than chance.  

---

## 2.1 Activation & Co-activation Matrices

- **Corpus & tokens**  
  We stream 3 000 000 raw tokens from **Bingsu/openwebtext_20p**, then keep every 5th token.  
  This yields $N\approx600\,000$ tokens.

- **Activation matrix**  
  $$  
    A\in\{0,1\}^{N\times F},  
    \quad A_{t,f} = 
      \begin{cases}
        1, &\text{if latent }f\text{ is in token }t\text{’s top-}k\text{ activations},\\
        0, &\text{otherwise}.
      \end{cases}
  $$

- **Per-feature counts**  
  $$  
    n_f = \sum_{t=1}^N A_{t,f}
    \quad\bigl(\text{how often latent }f\text{ fires}\bigr).
  $$

- **Co-activation matrix**  
  $$  
    C = A^\top A \in\mathbb Z^{F\times F},  
    \quad
    C_{i,j} = \sum_{t=1}^N A_{t,i}\,A_{t,j},
  $$  
  the number of tokens where latents $i$ and $j$ fire together.

---

## 2.2 Pairwise Statistics

Let  
- $n_i$ and $n_j$ be the activation counts of features $i,j$,  
- $n_{ij}=C_{ij}$ their co-activation count,  
- $N$ the total number of tokens,  
- $p_i = n_i/N,\;p_j = n_j/N,\;p_{ij} = n_{ij}/N$.

1. **Jaccard**  
   - Measures overlap of activation sets:  
     $$  
       J_{ij}
       = \frac{\lvert S_i \cap S_j\rvert}{\lvert S_i \cup S_j\rvert}
       = \frac{n_{ij}}{\,n_i + n_j - n_{ij}\,}\;\in[0,1],
     $$  
     where $S_i=\{t : A_{t,i}=1\}$, $n_i=|S_i|$ and $n_{ij}=|S_i\cap S_j|\,$.  

2. **Pointwise Mutual Information (PMI)**  
   - Let $n_i=\sum_{t}A_{t,i}$ and $n_{ij}=C_{ij}$, with total tokens $N$. Define  
     $$p_i = \frac{n_i}{N},\quad p_{ij} = \frac{n_{ij}}{N}.$$  
   - Then  
     $$  
       \mathrm{PMI}_{ij} = \log\frac{p_{ij}}{\,p_i\,p_j\,}\,,  
     $$  
     which is positive when features co-activate more often than expected under independence.  

3. **Normalized PMI (NPMI)**  
   - Scales PMI to $[-1,1]$ by its maximal possible value $-\log p_{ij}$:  
     $$  
       \mathrm{NPMI}_{ij} = \frac{\mathrm{PMI}_{ij}}{-\log p_{ij}}\;\in[-1,1].  
     $$  
   - Positive $\mathrm{NPMI}_{ij}$ indicates above-chance co-activation; negative indicates avoidance.

---

## 2.3 Cluster-level Averages  

For each semantic cluster $C=\{f_{1},\dots,f_{k}\}$ with $k\ge4$ features, we compute two aggregate co-activation scores:

1. **Average Jaccard**  
   We average all $\binom{k}{2}$ pairwise Jaccard indices $J_{ij}$ within $C$:  
   $$  
     \overline{J}(C)
     = \frac{1}{\binom{k}{2}}
       \sum_{1 \le i < j \le k} J_{f_{i}f_{j}}
     = \frac{2}{k(k-1)}
       \sum_{i<j\in C} J_{ij}\,.  
   $$  
   A high value of $\overline{J}(C)$ means that most feature pairs in the cluster co-activate frequently, indicating a tight functional grouping.

2. **Average NPMI**  
   We average the normalized pointwise mutual information across the same pairs:  
   $$  
     \overline{\mathrm{NPMI}}(C)
     = \frac{2}{k(k-1)}
       \sum_{i<j\in C} \mathrm{NPMI}_{ij}\,.  
   $$  
   Positive $\overline{\mathrm{NPMI}}(C)$ indicates that features co-occur more often than expected by chance; values near zero suggest co-activations at random levels.

By condensing all pairwise scores into $\overline{J}(C)$ and $\overline{\mathrm{NPMI}}(C)$, we obtain concise measures of cluster coherence across the corpus.  
 


---

## 2.4 Relative Score

We test whether a cluster’s average co-activation $\overline M(C)$ (either Jaccard or NPMI) exceeds chance:

1. **Background sampling**  
   - For each cluster of size $k$, draw 50 random subsets of $k$ features (out of $F$ total).  
   - Compute the cluster-level metric for each subset: $\overline M_{\rm bg}^{(s)}$, $s=1,\dots,50$.

2. **Background mean**  
   $$  
     \mu_{\rm bg}
     = \frac{1}{50}\sum_{s=1}^{50}\overline M_{\rm bg}^{(s)}.
   $$

3. **Relative ratio**  
   $$  
     r(C)
     = \frac{\overline M(C)}{\mu_{\rm bg}}.
   $$  
 
   - **Interpretation:**  
     - $r>1$: cluster co-activations are stronger than random.  
     - $r\approx1$: indistinguishable from random.  
     - $r<1$: weaker than chance.

4. **Layer summary**  
   - Over all clusters in a given layer, report the **median** ratio and the **10% trimmed mean** of $r$ to mitigate outliers.  

---

## Takeaways

- **Relative ratios > 1 across all layers** confirm that text-embedding clusters co-activate significantly more than random feature sets.  
- **Jaccard ratios often ≫ 1**, showing that features within a cluster fire together far more than chance.  
- **NPMI ratios > 1** indicate that mutual information among cluster features exceeds background expectation.  
- These results validate that clustering on **textual descriptions** yields functionally coherent SAE feature groups at every layer.  

---

# 3 SAE Match

SAE Match computes an optimal bijection $P_{l\to l+1}\in S_F$ between the $F=32\,768$ latent dimensions of layers $l$ and $l+1$, where $S_F$ denotes the symmetric group on $F$ elements. By selecting $P_{l\to l+1}$ to minimise a chosen dissimilarity metric, we obtain a guaranteed optimal alignment that lets us trace each feature’s persistence, splitting, or merging as representations evolve through the network.  



---

## 3.1 Latent Alignment as a Linear Assignment

Let 
- $\mathbf W^{(l)} = [\,\mathbf w_1^{(l)},\dots,\mathbf w_F^{(l)}]^\top\in\mathbb R^{F\times d}$ 
- $\mathbf W^{(l+1)} = [\,\mathbf w_1^{(l+1)},\dots,\mathbf w_F^{(l+1)}]^\top\in\mathbb R^{F\times d}$  
be the decoder‐weight matrices at layers $l$ and $l+1$.  Define a cost matrix $C\in\mathbb R^{F\times F}$ by  
$$
c_{ij} \;=\; d\bigl(\mathbf w_i^{(l)},\,\mathbf w_j^{(l+1)}\bigr),
$$  
where $d(\cdot,\cdot)$ is a chosen dissimilarity (e.g. $1-\cos$ or $\|\cdot\|_2^2$).  Then the alignment is the permutation  
$$
P_{l\to l+1}
= \arg\min_{P\in S_F}\sum_{i=1}^F c_{i,P(i)},
$$  
where $S_F$ is the symmetric group on $F$ elements.  The optimal $P_{l\to l+1}$ gives a globally minimal one‐to‐one mapping between the two latent spaces.  

---

## 3.2 Scalable Exact Matching

To solve the full assignment  
$$\min_{P\in S_F}\sum_{i=1}^F c_{i,P(i)},$$  
without forming an $F\times F$ cost matrix, we combine two steps:

1. **k-NN pruning**  
   - For each latent index $i\in\{1,\dots,F\}$ in layer $l$, compute distances $c_{ij}$ only to its $K=256$ nearest neighbours $j$ in layer $l+1$.  
   - This yields at most $F\cdot K$ candidate edges instead of $F^2$, reducing memory from $\mathcal O(F^2)$ to $\mathcal O(FK)$ and distance computations from $F^2$ to $F K$.

2. **Minimum‐Cost Flow assignment**  
   - Represent the pruned bipartite graph as a flow network with:  
     - A source connected to each “left” node $i$ with supply = 1.  
     - Each “right” node $j$ connected to the sink with demand = 1.  
     - Arcs $(i\to j)$ for each candidate edge, capacity = 1, cost = $c_{ij}$.  
     - Dummy self-arcs $(i\to i)$ with a large cost to ensure feasibility when true edges are missing.  
   - Run an exact Min-Cost Flow solver (OR-Tools) to send one unit of flow per latent, guaranteeing a perfect permutation.  
   - Overall complexity is dominated by $\mathcal O(FK\log(FK))$ flow operations, allowing each layer‐pair alignment in ~20 min on a modern GPU.

This approach retains exact optimality over the sparse edge set while scaling linearly in $F$ rather than quadratically.  

---

## 3.3 Matching Metrics

We compute a sparse cost graph on candidate edges $(i\to j)$ and then solve  
$$
P_{l\to l+1} = \arg\min_{P\in S_F}\sum_{i=1}^F c_{i,P(i)}.
$$  
The three cost functions are:

| metric | cost $c_{ij}$                                           | space               |
|:------:|:---------------------------------------------------------|:--------------------|
| **cos**  | $1 - \cos(\mathbf w_i,\mathbf w'_j)$                   | decoder columns     |
| **mse**  | $\|\mathbf w_i-\mathbf w'_j\|_2^2$                     | decoder columns     |
| **text** | $1 - \cos(\mathbf e_i,\mathbf e'_j)\;+\;\lambda\,\mathbf1_{\text{no-desc}}$ | text embeddings     |

---

### Cosine & MSE

- **cos** uses orientation only: $c_{ij}=1-\cos(\mathbf w_i,\mathbf w'_j)$.  
- **mse** uses squared Euclidean distance: $c_{ij}=\|\mathbf w_i-\mathbf w'_j\|_2^2$.  

Both operate on the $F\times d_{\rm model}$ decoder weight rows.

---

### Text‐Embedding Metric

1. **Unique-description embeddings**  
   - From `clusters_all_layers/layerXX_clusters.csv`, extract the $U$ unique processed_text strings.  
   - Encode each with SBERT to get  
     $$E_{\rm unique}\in\mathbb R^{U\times d_{\rm text}}.$$

2. **Broadcast to full latent set**  
   - Load `layerXX_full_labels.csv` with `rep_idx[f]` mapping each of the $F$ latents to a unique row $r\in[0,U)$ or –1 if no description.  
   - Form  
     $$E_{\rm full}[f] = 
       \begin{cases}
         E_{\rm unique}[\,\mathrm{rep\_idx}[f]\,], &\mathrm{if}\;\mathrm{rep\_idx}[f]\ge0,\\
         \mathbf0, &\mathrm{otherwise}.
       \end{cases}
     $$  
   - Record $\mathrm{no\_desc}[f]=1$ if `rep_idx[f]==–1`.

3. **Tie-breaking jitter & normalization**  
   - Detect exact-duplicate rows in $E_{\rm full}$ and add $\mathcal N(0,\sigma_{\rm jitter})$ noise to break ties.  
   - L2-normalize each row:  
     $$E_{\rm full}[f]\leftarrow \frac{E_{\rm full}[f]}{\|E_{\rm full}[f]\|_2}.$$

4. **Penalty for no-description**  
   - For any edge $(i\to j)$ where either $\mathrm{no\_desc}[i]=1$ or $\mathrm{no\_desc}[j]=1$, add a fixed cost $\lambda=0.3$.

5. **Sparse k-NN graph in text-space**  
   - Compute $1-\cos(E_{\rm full}^{(l)}[i],\,E_{\rm full}^{(l+1)}[j])$ and retain the top $K=256$ lowest-cost neighbours per $i$.  
   - Define  
     $$  
       c_{ij}
       = 1 - \cos(E_i^{(l)},E_j^{(l+1)}) \;+\;\lambda\,\mathbf1_{\text{no-desc}}.
     $$

6. **Exact assignment**  
   - Solve the reduced LAP via Min‐Cost Flow to obtain a bijection $P_{l\to l+1}$ that minimizes the total cost over these sparse edges.

This **text** metric enforces semantic consistency: latents with more similar textual descriptions are paired preferentially, while those lacking any description incur a penalty.  


---

## 3.4 Explained-Variance Diagnostic

We verify that our permutation $P_{l\to l+1}$ preserves **functional content** by measuring how well SAE$_l$ codes can reconstruct the true Transformer residuals at layer $l+1$.

1. **Collect residuals**  
   - Sample $W$ disjoint text windows of length $T$ from the token stream.  
   - For each window, record the post-residual hidden states  
     $$  
       H_l,\;H_{l+1}\;\in\;\mathbb R^{W\times d_{\rm model}}\,.  
     $$

2. **Encode–Permute–Decode**  
   - Encode $H_l$ with SAE$_l$:  
     $$  
       Z = \mathrm{SAE}_{l}^{\rm enc}(H_l)\;\in\;\mathbb R^{W\times F}\,.  
     $$  
   - Permute latent columns by $P$:  
     $$  
       Z_P[:,j] = Z[:,P^{-1}(j)]\,.  
     $$  
   - Decode with SAE$_{l+1}$ into the residual space of layer $l+1$:  
     $$  
       \widehat{H}_{l+1}
       = \mathrm{SAE}_{l+1}^{\rm dec}(Z_P)\;\in\;\mathbb R^{W\times d_{\rm model}}\,.  
     $$

3. **Explained-variance ratio**  
   - Compute the total variance of the prediction error versus true residuals:  
     $$  
       \mathrm{EV}
       = \frac{\mathrm{Var}\bigl(\widehat{H}_{l+1}-H_{l+1}\bigr)}
              {\mathrm{Var}\bigl(H_{l+1}\bigr)}\;\;(\ge0),  
     $$  
   - Define the reconstruction score  
     $$  
       R^2 = 1 - \mathrm{EV}\;\;\bigl(\le1\bigr).  
     $$

4. **Interpretation**  
   - **$R^2\approx1$** (EV$\approx0$): almost perfect recovery of layer $l+1$—the permutation fully captures functional mapping.  
   - **$R^2\approx0$** (EV$\approx1$): no predictive power—matched latents fail to reconstruct the next layer.
  
## 3.5 Key Observations

- **Early‐layer instability:**  
  In the first transformer layers, all metrics yield high EV ratios, indicating that feature permutations in these shallow layers do not reliably reconstruct the next‐layer residuals. Consequently, SAE Match in the initial layers is **unreliable** for tracing feature evolution.

- **Cosine vs. MSE alignment:**  
  The `cos` and `mse` metrics produce virtually **identical** permutations—geometric continuity in decoder‐weight space is captured equally well by orientation (cosine) or squared distance (L2).

- **Text vs. geometric metrics:**  
  Although the **explained‐variance curves** for `cos`, `mse`, and `text` are almost **indistinguishable** (mean EV ratios within ±0.02), the actual `text`‐based permutation differs from the geometric ones.

---

# 4 Evolution of Clusters via SAE Match

By aligning latents between layers with permutation $P_{l\to l+1}$, we can follow each semantic cluster’s fate: which features stay together, which fragments into multiple clusters, which clusters coalesce, and which features drop out (“leak”).  This analysis reveals the stability and transformation of high-level concepts as the model processes text, enabling us to pinpoint depths where representation structure shifts most dramatically.  

---

## 4.1 Inter‐layer Transition Matrices

For each layer $l$, let  
- $s_a$ = size of source cluster $a\in\{1,\dots,K_c\}$,  
- $\pi(i)=P_{l\to l+1}(i)$ the matched latent in layer $l+1$,  
- $c_i$ and $c'_j$ their cluster labels.  

We build the count matrix $N\in\mathbb Z^{K_c\times K_n}$:  
$$  
N_{a b}
= \bigl|\{\,i: c_i=a,\;c'_{\pi(i)}=b\}\bigr|.
$$  
The **leak** of source cluster $a$ is  
$$  
\ell_a
=1-\frac{\sum_{b}N_{a b}}{s_a},
$$  
i.e. the fraction of latents that map to “no cluster” in the next layer.  

---

## 4.2 Split, Merge, Survive & Leak

Starting from the dense transition counts $N_{a b}$ (source clusters $a=1..K_c$ to target clusters $b=1..K_n$) and source sizes $s_a$, we form the **row‐normalized flow**  
$$  
M_{a b} = \frac{N_{a b}}{s_a}\,,
$$  
so that $\sum_b M_{a b} = 1 - \ell_a$ and $\ell_a$ is the “leak” of cluster $a$ (fraction of latents that map to no cluster).

We then define:

- **Split fraction**  
  The share of source clusters that **fragment** into two or more downstream clusters:  
  - Count how many $b$ satisfy $M_{a b}\ge T_{\rm split}$ for each $a$.  
  - A cluster “splits” if it has at least two such $b$.
  
  $$\text{Split fraction} = \frac{\#\{a:\#\{b:M_{a b}\ge T_{\rm split}\}\ge2\}}{K_c}$$.

- **Merge fraction**  
  The share of target clusters that **merge** inputs from two or more sources:  
  - For each $b$, count how many $a$ satisfy $M_{a b}\ge T_{\rm split}$.  
   $$\text{Merge fraction} = \frac{\#\{b:\#\{a:M_{a b}\ge T_{\rm split}\}\ge2\}}{K_n}$$.

- **Survive fraction**  
  The proportion of source clusters that **persist** strongly into a single downstream cluster:  
  $$  
    \text{Survive fraction}
    = \frac{\#\{a: \max_b M_{a b}\ge T_{\rm surv}\}}{K_c}.
  $$

- **Mean leak**  
  The average fraction of features that **lose** any semantic assignment:  
  $$  
    \overline{\ell}
    = \frac1{K_c}\sum_{a=1}^{K_c}\ell_a.
  $$

Typical thresholds are $T_{\rm split}=0.20$ (at least 20% of mass) and $T_{\rm surv}=0.50$ (majority mass).  Plotting these four curves over layers highlights depths where clusters **fragment** (high split), **converge** (high merge), **persist** (high survive), or **disappear** to noise of clustering (high leak).  

---

## 4.3 Entropy & Mutual Information

To quantify how predictable and informative the cluster transitions are, we convert the raw counts $N_{a b}$ and leak rates $\ell_a$ into probability distributions:

1. **Conditional distribution**  
   For each source cluster $a$, ignore leaked mass and normalize:  
   $$  
     T_{a b}
     = \frac{N_{a b}}{\sum_{b=1}^{K_n}N_{a b}},  
     \quad\sum_{b=1}^{K_n}T_{a b}=1.
   $$

2. **Augmented distribution**  
   Include leak as an extra “target” bin so rows sum to one:  
   $$
     \widetilde M_{a b} =
     \begin{cases}
       \dfrac{N_{a b}}{s_a}, & b=1,\dots,K_n,\\[6pt]
       \ell_a,               & b=K_n+1,
     \end{cases}
     \quad\sum_{b=1}^{K_n+1}\widetilde M_{a b}=1.
   $$

3. **Row entropies**  
   Measure uncertainty of each source cluster’s transitions:  
   $$
     H_{\rm cond}(a)
     = -\sum_{b=1}^{K_n}T_{a b}\,\log T_{a b},  
     \quad
     H_{\rm aug}(a)
     = -\sum_{b=1}^{K_n+1}\widetilde M_{a b}\,\log\widetilde M_{a b}.
   $$

4. **Mutual information**  
   Treat source clusters $A$ as a random variable with marginal $p_a$ and targets $B$ or $B_{\rm aug}$ with marginals $p_b$ or $q_b$.  Then  
   $$
     I_{\rm cond}
     = \sum_{a=1}^{K_c}\sum_{b=1}^{K_n}
       p_a\,T_{a b}\,\log\frac{T_{a b}}{p_b},  
     \quad
     I_{\rm aug}
     = \sum_{a=1}^{K_c}\sum_{b=1}^{K_n+1}
       p_a\,\widetilde M_{a b}\,\log\frac{\widetilde M_{a b}}{q_b}.
   $$

- **Interpretation:**  
  - $H_{\rm cond}(a)$ close to 0 ⇒ cluster $a$ maps almost deterministically to one downstream cluster.  
  - Higher $I_{\rm cond}$ or $I_{\rm aug}$ indicates that knowing the source cluster greatly reduces uncertainty about its downstream assignment.  

---

### Takeaways

- **Leak** peaks in early layers → many features lose semantic identity.  
- **Split/merge** rates dip mid‐network, rise in deeper layers → conceptual reconfiguration near output.  
- **Entropies** decrease then plateau, while **MI** remains high ($\sim5$ bits), confirming overall cluster coherence despite structural changes.  

---

# 5 Evolution of Clusters via Optimal Transport

Whereas the permutation‐based framework in Section 4 enforces a rigid one‐to‐one mapping of individual latents, semantic clusters frequently undergo *partial* splits and merges.  To capture these *soft* transitions, we formulate a discrete **Optimal Transport (OT)** problem over cluster masses, yielding a fractional transport plan that precisely quantifies how cluster semantics distribute, merge, or leak between successive layers.

---

## 5.1 OT Formulation  

For two consecutive layers $l\to l+1$, let  
- $w_{a}$ be the mass (size) of source cluster $a=1,\dots,K_c$,  
- $v_{b}$ be the mass of target cluster $b=1,\dots,K_n$,  
- $C\in\mathbb R^{K_c\times K_n}$ be the cost matrix with entries $C_{ab}$ measuring dissimilarity between cluster centroids.

We seek a nonnegative transport plan $F\in\mathbb R_{\ge0}^{K_c\times K_n}$ that minimises the total mapping cost under exact mass conservation:

$$
F^\star
= \arg\min_{F\ge0}
  \sum_{a=1}^{K_c}\sum_{b=1}^{K_n}F_{ab}\,C_{ab}
\quad\text{s.t.}\quad
\sum_{b=1}^{K_n}F_{ab}=w_{a},\quad
\sum_{a=1}^{K_c}F_{ab}=v_{b}.
$$

In practice, we solve this using the entropic‐regularised **Sinkhorn** algorithm (with options for exact EMD or unbalanced variants).  The optimal plan $F^\star_{ab}$ then quantifies *how much* of source cluster $a$ flows into target cluster $b$, providing a continuous, soft alignment of semantic clusters across layers.  

---

## 5.2 Cluster Centroids & Transport Costs

- **Decoder centroid** for cluster $a$:  
  $$  
    \mu_a^{\rm dec}
    = \frac{1}{|C_a|}\sum_{i\in C_a}w_i^{\rm dec}\in\mathbb R^{d_{\rm model}}.  
  $$

- **Text centroid** for cluster $a$:  
  $$  
    \mu_a^{\rm txt}
    = \mathrm{normalize}\Bigl(\frac{1}{|C_a|}\sum_{i\in C_a}e_i\Bigr)\in\mathbb R^{d_{\rm txt}}.  
  $$

We define the pairwise cost matrix $C_{ab}$ between source $a$ and target $b$ as:

| Metric         | Cost $C_{ab}$                                                                 |
|:--------------:|:-------------------------------------------------------------------------------|
| decoder‐cosine | $1 - \cos\bigl(\mu_a^{\rm dec},\,\mu_b^{\rm dec}\bigr)$                         |
| decoder‐MSE    | $\|\mu_a^{\rm dec}-\mu_b^{\rm dec}\|_2^2$                                       |
| text‐cosine    | $1 - \cos\bigl(\mu_a^{\rm txt},\,\mu_b^{\rm txt}\bigr)$                         |

**Noise handling:** clusters with IDs –1 or –2 are merged into a single “noise”.  To penalize leakage into noise, one may add a constant shift to the corresponding costs $C_{a,\text{noise}}$.  


---

## 5.3 Scalability Strategies

When clusters number in the hundreds, the full cost matrix $C\in\mathbb R^{K_c\times K_n}$ can be too large to store or process efficiently.  We employ two key techniques:

1. **k-NN Sparsification**  
   - **Goal:** reduce memory and computation from $\mathcal{O}(K_cK_n)$ to $\mathcal{O}(K_c\,k)$.  
   - **Method:** for each source cluster $a$, we identify its $k$ smallest costs $C_{a,b}$ (via a row-wise $\texttt{argpartition}$).  We then zero out all other columns before calling the OT solver.  
   - **Effect:** only $K_c\times k$ edges remain, preserving local similarity structure while slashing both storage and per‐iteration complexity in Sinkhorn or EMD.

2. **Entropic Regularization & Float32 Precision**  
   - **Sinkhorn Algorithm:** we solve  
     $$
       F^\star \approx \arg\min_{F\ge0}
       \sum_{a,b}F_{ab}C_{ab} \;+\;\varepsilon\,\sum_{a,b}F_{ab}\log F_{ab},
     $$  
     where $\varepsilon>0$ controls smoothness.  
   - **Median‐Scaled $\varepsilon$:** set  
     $$\varepsilon = \alpha\times\mathrm{median}(C_{ab}),$$  
     with $\alpha\approx0.05$, so the regularizer adapts to the typical cost scale in each layer.  
   - **Float32 Computation:** by carrying both $C$ and the iterative Sinkhorn updates in 32-bit precision, GPU memory usage and bandwidth are cut nearly in half, with negligible impact on convergence or numerical stability.

Together, these strategies enable us to compute soft cluster‐to‐cluster transport plans across all 11 layer pairs in minutes rather than hours, even on large‐scale models.  


---

## 5.4 Export & Integration

Once the full OT plan $F\in\mathbb R^{K_c\times (K_n+1)}$ (including the “noise” column) is computed, we convert it into the **exact same** transition artifacts used in Section 4:

1. **Semantic → semantic counts**  
   - Discard the noise column and take the raw flows  
     $$F_{ab},\quad a=1..K_c,\;b=1..K_n.$$  
   - Round to the nearest integer:  
     $$N_{ab} = \mathrm{round}\bigl(F_{ab}\bigr).$$  
   - Save as a sparse NPZ file (`rows, cols, vals, shape`) at  
     `results/transitions_2step_ot/<tag>/trans_Lℓℓ+1.npz`.

2. **Leak vector**  
   - Extract the flow into the noise bin for each source cluster:  
     $$\ell_a = \frac{F_{a,\text{noise}}}{w_a}\,,\quad a=1..K_c.$$  
   - Save as `leak_Lℓ.npy` in the same folder.

Because these outputs exactly mirror the format produced by the permutation‐based pipeline, the downstream **split/merge**, **entropy**, and **MI** analyses can be applied unchanged.  This seamless interoperability allows direct head-to-head comparison of **hard** (permutation) vs. **soft** (OT) cluster evolution.  

---

## 5.5 Interpretation

- **Fractional migration:**  
  The transport plan entries $F_{ab}$ directly measure the proportion of source cluster $a$ that moves into each downstream cluster $b$.  This continuous view reveals partial splits and merges that a hard permutation would obscure.

- **Semantic attrition:**  
  A positive leak $\ell_a>0$ indicates that some fraction of cluster $a$ has no semantic counterpart in layer $l+1$, signifying concept disappearance or re-emergence.

- **Gradual vs. abrupt change:**  
  By comparing OT‐based metrics (entropy, mutual information, split/merge rates) with permutation‐based results, we can classify layers where cluster evolution is **gradual** — mass diffuses broadly with high transition entropy — versus **abrupt** — mass concentrates in a few dominant flows, reflecting crisp splits or merges.